In [1]:
import pandas as pd
import os
import numpy as np
#import scikit-learn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
folder_path = "/content/drive/MyDrive/UMich/"
#train_df = dd.read_csv(os.path.join(folder_path, "train_df_name_enc.csv"))
#test_df = dd.read_csv(os.path.join(folder_path, "test_df_name_enc.csv"))

In [5]:
os.path.exists(folder_path)
df = pd.read_csv(folder_path + "final_dfSI670.csv")

<ipython-input-5-3c84cf4670b8>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder_path + "final_dfSI670.csv")


In [6]:
desc_clusters = pd.read_csv(folder_path + "df_description_clustering.csv")
desc_clusters.head()

,description,birch_cluster
0,Lab Test - Identification of DNA from 22 Patho...,62682
1,ROOM & BOARD - PRIVATE MEDICAL & SURGICAL,67343
2,ROOM & BOARD - PRIVATE OBSTETRICAL,54333
3,ROOM & BOARD - PRIVATE HOSPICE,67343
4,ROOM & BOARD - SEMI-PRIVATE MEDICAL & SURGICAL,67343


In [8]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', "internal_revenue_code", "code_disambiguator", "name", "address", "city", "state", "units"])
df.head()

,cert_num,name,address,city,state,zip5,beds,payer,code,internal_revenue_code,...,code_disambiguator,BCBS,Medicare,Medicaid,UHC,Aetna,Cigna,Cash Price,Gross Charge,HospitalInsurance
0,230005,PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,5640 N ADRIAN HIGHWAY,ADRIAN,MI,49221,88,CASH PRICE,11770,NONE,...,NONE,0,0,0,0,0,0,1,0,0
1,230005,PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,5640 N ADRIAN HIGHWAY,ADRIAN,MI,49221,88,GROSS CHARGE,11770,NONE,...,NONE,0,0,0,0,0,0,0,1,0
2,230005,PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,5640 N ADRIAN HIGHWAY,ADRIAN,MI,49221,88,CASH PRICE,11771,NONE,...,NONE,0,0,0,0,0,0,1,0,0
3,230005,PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,5640 N ADRIAN HIGHWAY,ADRIAN,MI,49221,88,GROSS CHARGE,11771,NONE,...,NONE,0,0,0,0,0,0,0,1,0
4,230005,PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,5640 N ADRIAN HIGHWAY,ADRIAN,MI,49221,88,CASH PRICE,19120,NONE,...,NONE,0,0,0,0,0,0,1,0,0


In [14]:

#get all rows that have length code = 5 in df
df[df["code"].str.len() == 5][["code", "description"]].tail()

,code,description
659779,DRG99,NON-BACTERIAL INFECTION OF NERVOUS SYSTEM EXCE...
659780,DRG99,NON-BACTERIAL INFECTION OF NERVOUS SYSTEM EXCE...
659781,DRG99,NON-BACTERIAL INFECTION OF NERVOUS SYSTEM EXCE...
659782,DRG99,NON-BACTERIAL INFECTION OF NERVOUS SYSTEM EXCE...
659783,DRG99,NON-BACTERIAL INFECTION OF NERVOUS SYSTEM EXCE...


In [15]:
df["code"].isna().sum()

1817

In [16]:
df.dropna(subset=["code"], inplace=True)

In [17]:
df.shape

(657967, 26)

In [18]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', "internal_revenue_code", "code_disambiguator", "name", "address", "city", "state", "units"])
df.head()

,cert_num,zip5,beds,payer,code,description,inpatient_outpatient,price,BCBS,Medicare,Medicaid,UHC,Aetna,Cigna,Cash Price,Gross Charge,HospitalInsurance
0,230005,49221,88,CASH PRICE,11770,Removal of Pilonidal Cyst,OUTPATIENT,14266.79,0,0,0,0,0,0,1,0,0
1,230005,49221,88,GROSS CHARGE,11770,Removal of Pilonidal Cyst,OUTPATIENT,21948.91,0,0,0,0,0,0,0,1,0
2,230005,49221,88,CASH PRICE,11771,Removal of Pilonidal Cyst,OUTPATIENT,14266.79,0,0,0,0,0,0,1,0,0
3,230005,49221,88,GROSS CHARGE,11771,Removal of Pilonidal Cyst,OUTPATIENT,21948.91,0,0,0,0,0,0,0,1,0
4,230005,49221,88,CASH PRICE,19120,Removal of Breast Growth Requiring Incision,OUTPATIENT,15849.42,0,0,0,0,0,0,1,0,0


In [19]:
cash_df = df[df["payer"] == "CASH PRICE"][["cert_num", "description", "price"]].drop_duplicates()
cash_df.rename(columns={"price": "cash_price"}, inplace=True)
cash_df = pd.merge(cash_df, desc_clusters, on="description", how="inner")

In [20]:
gross_df = df[df["payer"] == "GROSS CHARGE"][["cert_num", "description", "price"]].drop_duplicates()
gross_df.rename(columns={"price": "gross_price"}, inplace=True)
gross_df = pd.merge(gross_df, desc_clusters, on="description", how="inner")
gross_df.head()

,cert_num,description,gross_price,birch_cluster
0,230005,Removal of Pilonidal Cyst,21948.91,70134
1,230005,Removal of Breast Growth Requiring Incision,24383.73,70079
2,230005,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",24132.75,54
3,230005,Repair of Shoulder Rotator Cuff,30462.49,70941
4,230005,Amputation Of Foot Or Toes,41709.26,66777


In [22]:
df_merge = pd.merge(cash_df, gross_df, on=["cert_num", "description", "birch_cluster"], how="inner")
df_merge.head()

,cert_num,description,cash_price,birch_cluster,gross_price
0,230005,Removal of Pilonidal Cyst,14266.79,70134,21948.91
1,230005,Removal of Breast Growth Requiring Incision,15849.42,70079,24383.73
2,230005,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",15686.29,54,24132.75
3,230005,Repair of Shoulder Rotator Cuff,19800.62,70941,30462.49
4,230005,Amputation Of Foot Or Toes,27111.02,66777,41709.26


In [54]:
df_final = df[(df["payer"] != "CASH PRICE") & (df["payer"] != "GROSS CHARGE")]
#df_final = pd.merge(df, df_merge, on=["cert_num", "description"], how="left")
#df_final.head()

In [56]:
df_final.reset_index(drop=True, inplace=True)
df_final.head()

,cert_num,zip5,beds,payer,code,description,inpatient_outpatient,price,BCBS,Medicare,Medicaid,UHC,Aetna,Cigna,Cash Price,Gross Charge,HospitalInsurance
0,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19120,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,0,0,0,0,0,0,0
1,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19125,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,0,0,0,0,0,0,0
2,230005,49221,88,BLUE CROSS BLUE SHIELD Commercial,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1731.03,1,0,0,0,0,0,0,0,0
3,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1624.22,0,1,0,0,0,0,0,0,0
4,230005,49221,88,BLUE CARE NETWORK Health Maintenance Organization,23410,Repair of Shoulder Rotator Cuff,OUTPATIENT,3741.87,1,0,0,0,0,0,0,0,0


In [57]:
df_final.shape

(599307, 17)

In [60]:
#for each instance of description in df_merge and cert_num that is in df, join on description and cert_num and add cash_price, birch_cluster, and gross_price to the row
df_final = pd.merge(df_final, df_merge[["description", "cert_num", "cash_price", "birch_cluster", "gross_price"]], on=["description", "cert_num"], how="inner")
df_final.head()


,cert_num,zip5,beds,payer,code,description,inpatient_outpatient,price,BCBS,Medicare,Medicaid,UHC,Aetna,Cigna,Cash Price,Gross Charge,HospitalInsurance,cash_price,birch_cluster,gross_price
0,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19120,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,0,0,0,0,0,0,0,15849.42,70079,24383.73
1,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19125,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,0,0,0,0,0,0,0,15849.42,70079,24383.73
2,230005,49221,88,BLUE CROSS BLUE SHIELD Commercial,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1731.03,1,0,0,0,0,0,0,0,0,15686.29,54,24132.75
3,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1624.22,0,1,0,0,0,0,0,0,0,15686.29,54,24132.75
4,230005,49221,88,BLUE CARE NETWORK Health Maintenance Organization,23410,Repair of Shoulder Rotator Cuff,OUTPATIENT,3741.87,1,0,0,0,0,0,0,0,0,19800.62,70941,30462.49


In [61]:
df_final.shape

(693558, 20)

In [62]:
df_final["gross_ratio"] = df_final["price"] / df_final["gross_price"]
df_final["cash_ratio"] = df_final["price"] / df_final["cash_price"]
df_final.head()

,cert_num,zip5,beds,payer,code,description,inpatient_outpatient,price,BCBS,Medicare,...,Aetna,Cigna,Cash Price,Gross Charge,HospitalInsurance,cash_price,birch_cluster,gross_price,gross_ratio,cash_ratio
0,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19120,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,...,0,0,0,0,0,15849.42,70079,24383.73,0.120741,0.185756
1,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19125,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,...,0,0,0,0,0,15849.42,70079,24383.73,0.120741,0.185756
2,230005,49221,88,BLUE CROSS BLUE SHIELD Commercial,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1731.03,1,0,...,0,0,0,0,0,15686.29,54,24132.75,0.071729,0.110353
3,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1624.22,0,1,...,0,0,0,0,0,15686.29,54,24132.75,0.067304,0.103544
4,230005,49221,88,BLUE CARE NETWORK Health Maintenance Organization,23410,Repair of Shoulder Rotator Cuff,OUTPATIENT,3741.87,1,0,...,0,0,0,0,0,19800.62,70941,30462.49,0.122835,0.188977


In [67]:
census_df = pd.read_csv(folder_path + "michigan_zipcode_data.csv")
census_df.head()
census_df.rename(columns={"ZIP_Code": "zip5"}, inplace=True)

In [68]:
df_final_census = pd.merge(df_final, census_df, on="zip5", how="inner")
df_final_census.shape

(693558, 33)

In [69]:
df_final_census.head()

,cert_num,zip5,beds,payer,code,description,inpatient_outpatient,price,BCBS,Medicare,...,Percentage_Over_65,Unemployment_Rate,Poverty_Rate,HS_rate,Bachelors_rate,Graduate_rate,With_Health_Insurance_Rate,Without_Health_Insurance_Rate,Labor_Force_Rate,Below_Poverty_Rate
0,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19120,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
1,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19125,Removal of Breast Growth Requiring Incision,OUTPATIENT,2944.12,0,1,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
2,230005,49221,88,BLUE CROSS BLUE SHIELD Commercial,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1731.03,1,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
3,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,20680,"Removal of Bone Implant (Screw, Wire, Pin, Nail)",OUTPATIENT,1624.22,0,1,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
4,230005,49221,88,BLUE CARE NETWORK Health Maintenance Organization,23410,Repair of Shoulder Rotator Cuff,OUTPATIENT,3741.87,1,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61


In [70]:
df_final_census = df_final_census.drop(columns=["description"])

In [71]:
#map outpatient inpatient unspecified to 1,2,3 in df_final_census
df_final_census["inpatient_outpatient"] = df_final_census["inpatient_outpatient"].map({"OUTPATIENT": 1, "INPATIENT": 2, "UNSPECIFIED": 3})
df_final_census.head()

,cert_num,zip5,beds,payer,code,inpatient_outpatient,price,BCBS,Medicare,Medicaid,...,Percentage_Over_65,Unemployment_Rate,Poverty_Rate,HS_rate,Bachelors_rate,Graduate_rate,With_Health_Insurance_Rate,Without_Health_Insurance_Rate,Labor_Force_Rate,Below_Poverty_Rate
0,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19120,1,2944.12,0,1,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
1,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,19125,1,2944.12,0,1,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
2,230005,49221,88,BLUE CROSS BLUE SHIELD Commercial,20680,1,1731.03,1,0,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
3,230005,49221,88,PRIORITY HEALTH MEDICARE ADVANTAGE,20680,1,1624.22,0,1,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61
4,230005,49221,88,BLUE CARE NETWORK Health Maintenance Organization,23410,1,3741.87,1,0,0,...,17.94,5.14,17.61,21.56,7.08,0.32,21.85,0.89,44.42,17.61


In [72]:
#one hot encode payer df_final_census
df_final_census = pd.get_dummies(df_final_census, columns=["payer"])
df_final_census.head()

,cert_num,zip5,beds,code,inpatient_outpatient,price,BCBS,Medicare,Medicaid,UHC,...,payer_UNITED MEDICAL RESOURCES WI,payer_United Healthcare,payer_United Healthcare Commercial,payer_United Healthcare Commercial HMO/PPO/ POS/EPO (ACA on/off Exchange),payer_United Healthcare Exclusive Provider Organization (EPO),payer_United Healthcare Medicare Advantage,payer_United Healthcare Preferred Provider Organization (PPO),payer_WELLCARE - Medicare Advantage,payer_WPS Insurance FOCUS (exclusively offers Aspirus Network),payer_WPS InsuranceFOCUS (exclusively offers Aspirus Network)
0,230005,49221,88,19120,1,2944.12,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
1,230005,49221,88,19125,1,2944.12,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
2,230005,49221,88,20680,1,1731.03,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,230005,49221,88,20680,1,1624.22,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
4,230005,49221,88,23410,1,3741.87,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False


In [74]:
df_final_census.to_csv(folder_path + "final_dfSI670_census_neuralnet.csv")

In [27]:
#start here
df_final_census = pd.read_csv(folder_path + "final_dfSI670_census_neuralnet.csv")

<ipython-input-27-4df11b6d4ec3>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final_census = pd.read_csv(folder_path + "final_dfSI670_census_neuralnet.csv")


In [28]:
df_final_census.head()

,Unnamed: 0,cert_num,zip5,beds,code,inpatient_outpatient,price,BCBS,Medicare,Medicaid,...,payer_UNITED MEDICAL RESOURCES WI,payer_United Healthcare,payer_United Healthcare Commercial,payer_United Healthcare Commercial HMO/PPO/ POS/EPO (ACA on/off Exchange),payer_United Healthcare Exclusive Provider Organization (EPO),payer_United Healthcare Medicare Advantage,payer_United Healthcare Preferred Provider Organization (PPO),payer_WELLCARE - Medicare Advantage,payer_WPS Insurance FOCUS (exclusively offers Aspirus Network),payer_WPS InsuranceFOCUS (exclusively offers Aspirus Network)
0,0,230005,49221,88,19120,1,2944.12,0,1,0,...,False,False,False,False,False,False,False,False,False,False
1,1,230005,49221,88,19125,1,2944.12,0,1,0,...,False,False,False,False,False,False,False,False,False,False
2,2,230005,49221,88,20680,1,1731.03,1,0,0,...,False,False,False,False,False,False,False,False,False,False
3,3,230005,49221,88,20680,1,1624.22,0,1,0,...,False,False,False,False,False,False,False,False,False,False
4,4,230005,49221,88,23410,1,3741.87,1,0,0,...,False,False,False,False,False,False,False,False,False,False


In [29]:
#change True and False Values to 1 and 0 df_final_census
df_final_census = df_final_census.replace({True: 1, False: 0})
df_final_census.head()

<ipython-input-29-9cc4d862f6e4>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_census = df_final_census.replace({True: 1, False: 0})


,Unnamed: 0,cert_num,zip5,beds,code,inpatient_outpatient,price,BCBS,Medicare,Medicaid,...,payer_UNITED MEDICAL RESOURCES WI,payer_United Healthcare,payer_United Healthcare Commercial,payer_United Healthcare Commercial HMO/PPO/ POS/EPO (ACA on/off Exchange),payer_United Healthcare Exclusive Provider Organization (EPO),payer_United Healthcare Medicare Advantage,payer_United Healthcare Preferred Provider Organization (PPO),payer_WELLCARE - Medicare Advantage,payer_WPS Insurance FOCUS (exclusively offers Aspirus Network),payer_WPS InsuranceFOCUS (exclusively offers Aspirus Network)
0,0,230005,49221,88,19120,1,2944.12,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,230005,49221,88,19125,1,2944.12,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,230005,49221,88,20680,1,1731.03,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,230005,49221,88,20680,1,1624.22,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,230005,49221,88,23410,1,3741.87,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
df_final_census["gross_cash_mean"] = (df_final_census["gross_price"] + df_final_census["cash_price"]) / 2
df_final_census = df_final_census.drop(columns=["gross_price", "cash_price", "cash_ratio", "code", "Unnamed: 0", "gross_ratio"])
df_final_census.to_csv(folder_path + "final_dfSI670_census_neuralnet2.csv")

In [31]:
df_final_census.head()

,cert_num,zip5,beds,inpatient_outpatient,price,BCBS,Medicare,Medicaid,UHC,Aetna,...,payer_United Healthcare,payer_United Healthcare Commercial,payer_United Healthcare Commercial HMO/PPO/ POS/EPO (ACA on/off Exchange),payer_United Healthcare Exclusive Provider Organization (EPO),payer_United Healthcare Medicare Advantage,payer_United Healthcare Preferred Provider Organization (PPO),payer_WELLCARE - Medicare Advantage,payer_WPS Insurance FOCUS (exclusively offers Aspirus Network),payer_WPS InsuranceFOCUS (exclusively offers Aspirus Network),gross_cash_mean
0,230005,49221,88,1,2944.12,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,20116.575
1,230005,49221,88,1,2944.12,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,20116.575
2,230005,49221,88,1,1731.03,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19909.520
3,230005,49221,88,1,1624.22,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,19909.520
4,230005,49221,88,1,3741.87,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25131.555


In [6]:
gross = df_final_census.drop(columns=["gross_price", "cash_price", "price", "cash_ratio", "code"])
gross.head()

,Unnamed: 0,cert_num,zip5,beds,inpatient_outpatient,BCBS,Medicare,Medicaid,UHC,Aetna,...,payer_UNITED MEDICAL RESOURCES WI,payer_United Healthcare,payer_United Healthcare Commercial,payer_United Healthcare Commercial HMO/PPO/ POS/EPO (ACA on/off Exchange),payer_United Healthcare Exclusive Provider Organization (EPO),payer_United Healthcare Medicare Advantage,payer_United Healthcare Preferred Provider Organization (PPO),payer_WELLCARE - Medicare Advantage,payer_WPS Insurance FOCUS (exclusively offers Aspirus Network),payer_WPS InsuranceFOCUS (exclusively offers Aspirus Network)
0,0,230005,49221,88,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,230005,49221,88,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,230005,49221,88,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,230005,49221,88,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,230005,49221,88,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
gross.to_csv(folder_path + "gross_neuralnet.csv")

In [19]:
list(df_final_census.columns)

['cert_num',
 'zip5',
 'beds',
 'inpatient_outpatient',
 'BCBS',
 'Medicare',
 'Medicaid',
 'UHC',
 'Aetna',
 'Cigna',
 'Cash Price',
 'Gross Charge',
 'HospitalInsurance',
 'birch_cluster',
 'Median_Income',
 'Percentage_Over_65',
 'Unemployment_Rate',
 'Poverty_Rate',
 'HS_rate',
 'Bachelors_rate',
 'Graduate_rate',
 'With_Health_Insurance_Rate',
 'Without_Health_Insurance_Rate',
 'Labor_Force_Rate',
 'Below_Poverty_Rate',
 'payer_AETNA',
 'payer_AETNA - Commercial',
 'payer_AETNA - PPO',
 'payer_AETNA ALLIED BENEFIT SYSTEMS [18109]',
 'payer_AETNA BETTER HEALTH MANAGED MEDICAID',
 'payer_AETNA BETTER HEALTH OF MICHIGAN - Medicaid Advantage',
 'payer_AETNA COMMERCIAL',
 'payer_AETNA GLOBAL BENEFITS',
 'payer_AETNA HMO POS [17005]',
 'payer_AETNA HTH WORLDWIDE',
 'payer_AETNA INSURANCE',
 'payer_AETNA INTERNATIONAL',
 'payer_AETNA LIFE INS',
 'payer_AETNA LIFE INS CO',
 'payer_AETNA MTU STUDENT INS',
 'payer_AETNA PPO',
 'payer_AETNA PRISON HEALTH SERVICES',
 'payer_AETNA STUDENT INS 

In [4]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
folder_path = "/content/drive/MyDrive/UMich/"
gross = pd.read_csv(folder_path + "gross_neuralnet.csv")

In [10]:
cash = df_final_census.drop(columns=["gross_price", "cash_price", "price", "gross_ratio", "code"])
cash.head()

,Unnamed: 0,cert_num,zip5,beds,code,inpatient_outpatient,BCBS,Medicare,Medicaid,UHC,...,payer_UNITED MEDICAL RESOURCES WI,payer_United Healthcare,payer_United Healthcare Commercial,payer_United Healthcare Commercial HMO/PPO/ POS/EPO (ACA on/off Exchange),payer_United Healthcare Exclusive Provider Organization (EPO),payer_United Healthcare Medicare Advantage,payer_United Healthcare Preferred Provider Organization (PPO),payer_WELLCARE - Medicare Advantage,payer_WPS Insurance FOCUS (exclusively offers Aspirus Network),payer_WPS InsuranceFOCUS (exclusively offers Aspirus Network)
0,0,230005,49221,88,19120,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,230005,49221,88,19125,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,230005,49221,88,20680,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,230005,49221,88,20680,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,230005,49221,88,23410,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

In [32]:
groups = df_final_census['cert_num']

In [34]:
# Grouped train/validation/test split
df_final_census_features = df_final_census.drop(columns=['price', 'cert_num'])
df_final_census_target = df_final_census['price']


gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_val_idx, test_idx = next(gss.split(df_final_census_features, df_final_census_target, groups=groups))

train_val_df = df_final_census.iloc[train_val_idx]
test_df = df_final_census.iloc[test_idx]

# Further split the train/validation set into train and validation
gss_val = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)  # 25% of train_val for validation
train_idx, val_idx = next(gss_val.split(train_val_df, train_val_df['price'], groups=train_val_df['cert_num']))

train_df = train_val_df.iloc[train_idx]
val_df = train_val_df.iloc[val_idx]

# Extract features and target for each split
X_train, y_train = train_df.drop(columns=["price"]), train_df["price"]
X_val, y_val = val_df.drop(columns=["price"]), val_df["price"]
X_test, y_test = test_df.drop(columns=["price"]), test_df["price"]

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Grouped train/validation/test split
gross_features = df_final_census.drop(columns=['gross_ratio'])
gross_target = gross['gross_ratio']

#cash_features = cash.drop(columns=['cash_ratio'])
#cash_target = cash['cash_ratio']

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_val_idx, test_idx = next(gss.split(gross_features, gross_target, groups=groups))

train_val_df = gross.iloc[train_val_idx]
test_df = gross.iloc[test_idx]

# Further split the train/validation set into train and validation
gss_val = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)  # 25% of train_val for validation
train_idx, val_idx = next(gss_val.split(train_val_df, train_val_df['gross_ratio'], groups=train_val_df['cert_num']))

train_df = train_val_df.iloc[train_idx]
val_df = train_val_df.iloc[val_idx]

# Extract features and target for each split
X_train, y_train = train_df.drop(columns=["gross_ratio"]), train_df['gross_ratio']
X_val, y_val = val_df.drop(columns=["gross_ratio"]), val_df['gross_ratio']
X_test, y_test = test_df.drop(columns=["gross_ratio"]), test_df['gross_ratio']

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Huber
from sklearn.metrics import mean_squared_error

# Define the model
def build_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')  # Regression output
    ])
    model.compile(optimizer='adam', loss= "mse", metrics=['mae'])
    return model

# Build and compile the model
model = build_model((X_train.shape[1],))

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping]
)

# Evaluate on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {loss}")
y_test_pred = model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {final_rmse:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.1489 - mae: 0.4362 - val_loss: 0.0207 - val_mae: 0.1437
Epoch 2/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1370 - mae: 0.3997 - val_loss: 0.0212 - val_mae: 0.1432
Epoch 3/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1337 - mae: 0.3922 - val_loss: 0.0203 - val_mae: 0.1419
Epoch 4/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1337 - mae: 0.3918 - val_loss: 0.0196 - val_mae: 0.1360
Epoch 5/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1324 - mae: 0.3891 - val_loss: 0.0199 - val_mae: 0.1356
Epoch 6/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1346 - mae: 0.3928 - val_loss: 0.0195 - val_mae: 0.1311
Epoch 7/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1368 - mae: 0.3969 - val_loss: 0.0195 - val_mae: 0.1323
Epoch 8/100
15438/15438 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 0.1356 - mae: 0.3946 - val_loss: 0.0214 - val_mae: 0.1495
Epoch 9/

In [14]:
gross["gross_ratio"].describe()

,gross_ratio
count,693558.000000
mean,0.719554
std,2.869635
min,0.000071
25%,0.273030
50%,0.444421
75%,0.721879
max,153.272333


In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Load dataset
# Assuming 'data' is a Pandas DataFrame containing your dataset
# Replace this with actual data loading
#data = pd.read_csv('your_dataset.csv')

# Separate features and target
X = df_final_census.drop(columns=['price', 'cert_num'])
y = df_final_census['price']
groups = df_final_census['cert_num']

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split data into train and test by group
train_idx, test_idx = train_test_split(np.arange(len(df_final_census)), test_size=0.2, stratify=groups, random_state=42)
X_train, X_test = X_normalized[train_idx], X_normalized[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
train_groups = groups.iloc[train_idx]

y_train = np.log1p(y_train)  # Apply log(1 + price) to avoid issues with zero prices
y_test = np.log1p(y_test)

# Define the model
def create_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1)  # Single output for price prediction
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Perform GroupKFold cross-validation
group_kfold = GroupKFold(n_splits=5)
cv_results = []
best_model = None
best_mse = float('inf')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for train_index, val_index in group_kfold.split(X_train, y_train, train_groups):
    X_tr, X_val = X_train[train_index], X_train[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Build and train the model
    model = create_model(X_tr.shape[1])
    model.fit(X_tr, y_tr, validation_data=(X_val, y_val), epochs=50, batch_size=32, verbose=0, callbacks=[early_stopping])

    # Evaluate on validation data
    val_predictions = np.expm1(model.predict(X_val))
    val_mse = mean_squared_error(y_val, val_predictions)
    cv_results.append(val_mse)

    # Save the best model
    if val_mse < best_mse:
        best_mse = val_mse
        best_model = model

print("Cross-validation MSEs:", cv_results)
print("Best Cross-validation MSE:", best_mse)

# Retrain on full training data with the best model configuration
best_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate on the test set
test_predictions = np.expm1(best_model.predict(X_test))
test_mse = mean_squared_error(y_test, test_predictions)
rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
print("Test MSE:", test_mse)
print("Test RMSE:", rmse)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3475/3475 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3467/3467 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3467/3467 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3466/3466 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3466/3466 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Cross-validation MSEs: [160941027.56997153, 153744277.18926334, 2897116855.5497875, 257915677.1841992, 5380281370.055601]
Best Cross-validation MSE: 153744277.18926334
Epoch 1/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.6587 - mae: 0.6164
Epoch 2/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.5032 - mae: 0.5239
Epoch 3/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.4403 - mae: 0.4803
Epoch 4/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.4152 - mae: 0.4622
Epoch 5/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 0.4065 - mae: 0.4545
Epoch 6/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 0.3983 - mae: 0.4487
Epoch 7/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 0.3994 - mae: 0.4489
Epoch 8/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - loss: 0.3935 - mae: 0.4449
Epoch 9/50
17339/17339 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3911 - mae: 0.4438
Epoch 10/50
173

In [38]:
best_model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 128)                 │          35,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 133,253 (520.52 KB)

 Trainable params: 44,289 (173.00 KB)

 Non-trainable params: 384 (1.50 KB)

 Optimizer params: 88,580 (346.02 KB)

In [40]:
df_final_census_filt = df_final_census[df_final_census["price"] < 35000]

In [42]:
df_final_census_filt["price"].describe()

,price
count,672128.000000
mean,5460.276805
std,6223.334747
min,3.310000
25%,1704.340000
50%,2947.500000
75%,6250.970000
max,34995.550000


In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Load dataset
# Assuming 'data' is a Pandas DataFrame containing your dataset
# Replace this with actual data loading
#data = pd.read_csv('your_dataset.csv')

# Separate features and target
X = df_final_census_filt.drop(columns=['price', 'cert_num'])
y = df_final_census_filt['price']
groups = df_final_census_filt['cert_num']

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split data into train and test by group
train_idx, test_idx = train_test_split(np.arange(len(df_final_census_filt)), test_size=0.2, stratify=groups, random_state=42)
X_train, X_test = X_normalized[train_idx], X_normalized[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
train_groups = groups.iloc[train_idx]

y_train = np.log1p(y_train)  # Apply log(1 + price) to avoid issues with zero prices
y_test = np.log1p(y_test)

# Define the model
def create_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1)  # Single output for price prediction
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Perform GroupKFold cross-validation
group_kfold = GroupKFold(n_splits=5)
cv_results = []
best_model = None
best_mse = float('inf')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for train_index, val_index in group_kfold.split(X_train, y_train, train_groups):
    X_tr, X_val = X_train[train_index], X_train[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Build and train the model
    model = create_model(X_tr.shape[1])
    model.fit(X_tr, y_tr, validation_data=(X_val, y_val), epochs=50, batch_size=32, verbose=0, callbacks=[early_stopping])

    # Evaluate on validation data
    val_predictions = np.expm1(model.predict(X_val))
    val_mse = mean_squared_error(y_val, val_predictions)
    cv_results.append(val_mse)

    # Save the best model
    if val_mse < best_mse:
        best_mse = val_mse
        best_model = model

print("Cross-validation MSEs:", cv_results)
print("Best Cross-validation MSE:", best_mse)

# Retrain on full training data with the best model configuration
best_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate on the test set
test_predictions = np.expm1(best_model.predict(X_test))
test_mse = mean_squared_error(y_test, test_predictions)
rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
print("Test MSE:", test_mse)
print("Test RMSE:", rmse)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3361/3361 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3360/3360 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3362/3362 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3361/3361 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3362/3362 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Cross-validation MSEs: [10305726.330723137, 23715086.291139904, 11664156.105780026, 31035503.228689786, 24088469.968396887]
Best Cross-validation MSE: 10305726.330723137
Epoch 1/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.4446 - mae: 0.4859
Epoch 2/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3973 - mae: 0.4522
Epoch 3/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.3809 - mae: 0.4396
Epoch 4/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3703 - mae: 0.4319
Epoch 5/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3688 - mae: 0.4293
Epoch 6/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.3678 - mae: 0.4283
Epoch 7/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3639 - mae: 0.4252
Epoch 8/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 0.3639 - mae: 0.4251
Epoch 9/50
16804/16804 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.3574 - mae: 0.4216
Epoch 10/50
1